In [1]:
import optuna
import os
import kaggle
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)


VERSION = 5
OUTPUT = f'extra_tree_clf_submisson_v{VERSION}.csv'

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

seed = 42
splits = 5

np.random.seed(seed)

In [5]:
partial_pipeline = make_pipeline(SimpleImputer(), FunctionTransformer(func=np.log1p, validate=False), StandardScaler())

In [6]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600, 100),
        'n_jobs': trial.suggest_categorical('n_jobs', [-1]),
        'random_state': trial.suggest_categorical('random_state', [seed]),
        'max_depth': trial.suggest_int('max_depth', 4, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 150),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 60),
        'criterion': trial.suggest_categorical("criterion", ["gini"]),
        'max_features': trial.suggest_categorical('max_features', ["sqrt", "log2"]),
    }

    pipeline = make_pipeline(partial_pipeline, ExtraTreesClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=splits, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, show_progress_bar=True)

[I 2023-10-14 17:26:50,029] A new study created in memory with name: no-name-06784de6-1647-4460-9137-0d9e113e1baa
Best trial: 0. Best value: 0.777077:   3%|█████▍                                                                                                                                                              | 1/30 [00:26<12:56, 26.77s/it]

[I 2023-10-14 17:27:16,793] Trial 0 finished with value: 0.7770765089285907 and parameters: {'n_estimators': 400, 'n_jobs': -1, 'random_state': 42, 'max_depth': 27, 'min_samples_split': 39, 'min_samples_leaf': 57, 'criterion': 'gini', 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7770765089285907.


Best trial: 1. Best value: 0.778301:   7%|██████████▉                                                                                                                                                         | 2/30 [00:47<10:46, 23.10s/it]

[I 2023-10-14 17:27:37,326] Trial 1 finished with value: 0.7783006270366193 and parameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 42, 'max_depth': 13, 'min_samples_split': 59, 'min_samples_leaf': 9, 'criterion': 'gini', 'max_features': 'log2'}. Best is trial 1 with value: 0.7783006270366193.


Best trial: 2. Best value: 0.778565:  10%|████████████████▍                                                                                                                                                   | 3/30 [01:38<16:09, 35.91s/it]

...

[I 2023-10-14 17:45:42,188] Trial 27 finished with value: 0.779883887933486 and parameters: {'n_estimators': 400, 'n_jobs': -1, 'random_state': 42, 'max_depth': 40, 'min_samples_split': 91, 'min_samples_leaf': 6, 'criterion': 'gini', 'max_features': 'log2'}. Best is trial 11 with value: 0.7825094059676625.


Best trial: 11. Best value: 0.782509:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 29/30 [19:39<00:47, 47.20s/it]

[I 2023-10-14 17:46:29,580] Trial 28 finished with value: 0.7787958038800475 and parameters: {'n_estimators': 600, 'n_jobs': -1, 'random_state': 42, 'max_depth': 47, 'min_samples_split': 110, 'min_samples_leaf': 18, 'criterion': 'gini', 'max_features': 'log2'}. Best is trial 11 with value: 0.7825094059676625.


Best trial: 11. Best value: 0.782509: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [20:16<00:00, 40.56s/it]

[I 2023-10-14 17:47:06,735] Trial 29 finished with value: 0.7785635138411242 and parameters: {'n_estimators': 400, 'n_jobs': -1, 'random_state': 42, 'max_depth': 36, 'min_samples_split': 26, 'min_samples_leaf': 10, 'criterion': 'gini', 'max_features': 'sqrt'}. Best is trial 11 with value: 0.7825094059676625.


In [7]:
study.best_params, study.best_value

({'n_estimators': 500,
  'n_jobs': -1,
  'random_state': 42,
  'max_depth': 50,
  'min_samples_split': 99,
  'min_samples_leaf': 1,
  'criterion': 'gini',
  'max_features': 'log2'},
 0.7825094059676625)

In [8]:
pipeline = make_pipeline(partial_pipeline, ExtraTreesClassifier(**study.best_params))
pipeline.fit(X, y)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('simpleimputer', SimpleImputer()),
                                 ('functiontransformer',
                                  FunctionTransformer(func=<ufunc 'log1p'>)),
                                 ('standardscaler', StandardScaler())])),
                ('extratreesclassifier',
                 ExtraTreesClassifier(max_depth=50, max_features='log2',
                                      min_samples_split=99, n_estimators=500,
                                      n_jobs=-1, random_state=42))])

In [9]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv(OUTPUT)